## 使用嵌入

这个笔记本包含一些有用的代码片段，可以通过OpenAI API使用`text-embedding-3-small`模型来嵌入文本。


In [ ]:
from openai import OpenAI
client = OpenAI()

embedding = client.embeddings.create(
    input="Your text goes here", model="text-embedding-3-small"
).data[0].embedding
len(embedding)



1536

建议使用'tenacity'包或其他指数退避实现来更好地管理API速率限制，因为过快地频繁调用API可能会触发速率限制。使用以下函数可以确保尽快获取嵌入。


In [ ]:
# 反面案例（缓慢且受限速）
from openai import OpenAI
client = OpenAI()

num_embeddings = 10000 # 一些较大的数字
for i in range(num_embeddings):
    embedding = client.embeddings.create(
        input="Your text goes here", model="text-embedding-3-small"
    ).data[0].embedding
    print(len(embedding))


In [2]:
# 最佳实践
from tenacity import retry, wait_random_exponential, stop_after_attempt
from openai import OpenAI
client = OpenAI()

# 最多重试6次，采用指数退避策略，起始延迟为1秒，最大延迟不超过20秒。
@retry(wait=wait_random_exponential(min=1, max=20), stop=stop_after_attempt(6))
def get_embedding(text: str, model="text-embedding-3-small") -> list[float]:
    return client.embeddings.create(input=[text], model=model).data[0].embedding

embedding = get_embedding("Your text goes here", model="text-embedding-3-small")
print(len(embedding))


1536
